In [2]:
# install moondream dependencies
!pip install --upgrade torch --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp310-cp310-linux_x86_64.whl (780.4 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.5.1+cu121 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.5.1+cu121 which is incompatible.


In [3]:
!pip install flash_attn # requires torch > 2.2

  Using cached flash_attn-2.7.4.post1.tar.gz (6.0 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for flash_attn: filename=flash_attn-2.7.4.post1-cp310-cp310-linux_x86_64.whl size=187797312 sha256=b267f80a08e516292cdd748056a2178a45b8abedf7fca123292eb17c21c8c87c
  Stored in directory: /root/.cache/pip/wheels/59/ce/d5/08ea07bfc16ba218dc65a3a7ef9b6a270530bcbd2cea2ee1ca
Successfully built flash_attn


In [16]:
"""
smolvlm-256m is a 0.25B text-to-image model that has several interesting capabilities trained in.
These include:
- captioning - with various lengths
- general querying (e.g., "how many people are in this image?")
- translate text on image

There are several smaller versions of the model as well.

the repo: https://huggingface.co/collections/HuggingFaceTB/smolvlm-256m-and-500m-6791fafc5bb0ab8acc960fb0
"""

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

# Automatically determine the best available device
if torch.backends.mps.is_available():
    device = "mps"  # Metal (Apple Silicon)
elif torch.cuda.is_available():
    device = "cuda"  # NVIDIA GPU
else:
    device = "cpu"  # Fallback to CPU

print('device is', device)

image_path = "./examples/both pills.jpeg"
image = Image.open(image_path)

# Initialize processor and model
model_size = "HuggingFaceTB/SmolVLM-256M-Instruct" # HuggingFaceTB/SmolVLM-500M-Instruct
processor = AutoProcessor.from_pretrained(model_size)
model = AutoModelForVision2Seq.from_pretrained(
    model_size,
    torch_dtype=torch.bfloat16,
    _attn_implementation="flash_attention_2" if device == "cuda" else "eager",
).to(device)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe this image?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image], return_tensors="pt")
inputs = inputs.to(device)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=250)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])


device is cuda


Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


User:



Can you describe this image?
Assistant: The image features a close-up of a man's face. The man is wearing dark sunglasses and has a serious expression on his face. His eyes are slightly narrowed, and his eyebrows are furrowed, suggesting a sense of contemplation or contemplation. The man's hair is short and appears to be bald, which is a common feature in many men.

The background of the image is blurred, which helps to focus the viewer's attention on the man's face. The background is somewhat blurred, which helps to emphasize the man's features and the blurred background.

At the top of the image, there is text that reads "Did you just take both pills?" in bold, white letters. The text is in a sans-serif font, which is easy to read and does not distract from the main subject of the image.

The image is likely a promotional or meme-like image, as it is designed to convey a message about the importance of taking both medications. The text at the top of the image is likely to gr